### Pirámides

In [1]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
# %matplotlib inline
%matplotlib
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg


In [2]:
# Cargamos la imagen a procesar
img = cv.imread('justiceleague.jpg')

#### 1. Bajamos resolución con pirámides (Gaussianas)

In [3]:
print('\nTamaño imagen: {} pix'.format(img.shape))

#Nivel 1
img_nivel_1 = cv.pyrDown(img)
print('\nTamaño imagen: {} pix'.format(img_nivel_1.shape))

#Nivel 2
img_nivel_2 = cv.pyrDown(img_nivel_1)
print('\nTamaño imagen: {} pix'.format(img_nivel_2.shape))

#Nivel 3
img_nivel_3 = cv.pyrDown(img_nivel_2)
print('\nTamaño imagen: {} pix'.format(img_nivel_3.shape))

# Muestro los resultados
cv.imshow("Imagen Original", img)
cv.imshow("Imagen Nivel 1", img_nivel_1)
cv.imshow("Imagen Nivel 2", img_nivel_2)
cv.imshow("Imagen Nivel 3", img_nivel_3)


Tamaño imagen: (1080, 1920, 3) pix

Tamaño imagen: (540, 960, 3) pix

Tamaño imagen: (270, 480, 3) pix

Tamaño imagen: (135, 240, 3) pix


#### 2. Subimos un nivel (desde el último)

In [4]:
# Esto aumenta la resolución inyectando filas y columnas de ceros intercaladas y luego 
# convolucionando el resultado con el núcleo gaussiano 5 × 5 multiplicado por 4
img_nivel_2 = cv.pyrUp(img_nivel_3)

# Muestro el resultado por subir el nivel
cv.imshow("Imagen Nivel 2", img_nivel_2)

#### Otra forma de modificar la resolucion (util para downscaling o upscaling con escala arbitraria)

In [5]:
img = cv.imread('origami.jpg')
print(f'Tamaño original: {img.shape}')

scales = np.linspace(0.5, 2, 5)
resized_imgs = list()

# generamos nuestra "piramide" de tamaños arbitrarios
print(f'Dimensiones generadas:')
for scale in scales:
    resized_imgs.append(cv.resize(img, None, fx=scale, fy=scale))
    print(resized_imgs[-1].shape)

# mostramos las imagenes
for i, img in enumerate(resized_imgs):
    cv.imshow(f'nivel-{i}', img)

Tamaño original: (360, 480, 3)
Dimensiones generadas:
(180, 240, 3)
(315, 420, 3)
(450, 600, 3)
(585, 780, 3)
(720, 960, 3)


#### 3. Pirámide Laplaciana

In [6]:
base = img.copy()

# Creamos la pirámide Gaussiana
piramide_gauss = [base]
for i in range(3):
    base = cv.pyrDown(base)
    piramide_gauss.append(base)

# El último nivel de la Gaussiana permanece en el mismo lugar en la Laplaciana
punta_laplaciana = piramide_gauss[-1]
 
# Creamos la pirámide Laplaciana
piramide_laplace = [punta_laplaciana]
for i in range(3,0,-1):
    size = (piramide_gauss[i - 1].shape[1], piramide_gauss[i - 1].shape[0])
    gaussiana_expandida = cv.pyrUp(piramide_gauss[i], dstsize=size)
    laplaciana = cv.subtract(piramide_gauss[i-1], gaussiana_expandida)
    piramide_laplace.append(laplaciana)
    cv.imshow('nivel-{}'.format(i-1),laplaciana)

#### 4. Fusión de imágenes

In [7]:
# Paso 2
#=======
# Encontrar las pirámides Gaussianas de las dos imágenes y la máscara
def gaussian_pyramid(img, num_levels):
    lower = img.copy()
    gaussian_pyr = [lower]
    for i in range(num_levels):
        lower = cv.pyrDown(lower)
        gaussian_pyr.append(np.float32(lower))
    return gaussian_pyr

# Paso 3
#=======
# Calcular la pirámide Laplaciana
def laplacian_pyramid(gaussian_pyr):
    laplacian_top = gaussian_pyr[-1]
    num_levels = len(gaussian_pyr) - 1
    
    laplacian_pyr = [laplacian_top]
    for i in range(num_levels,0,-1):
        size = (gaussian_pyr[i - 1].shape[1], gaussian_pyr[i - 1].shape[0])
        gaussian_expanded = cv.pyrUp(gaussian_pyr[i], dstsize=size)
        laplacian = np.subtract(gaussian_pyr[i-1], gaussian_expanded)
        laplacian_pyr.append(laplacian)
    return laplacian_pyr

# Paso 4
#=======
# Fusionar las dos imagenes según la máscara indicada
def blend(laplacian_A,laplacian_B,mask_pyr):
    LS = []
    for la,lb,mask in zip(laplacian_A,laplacian_B,mask_pyr):
        ls = lb * mask + la * (1.0 - mask)
        LS.append(ls)
    return LS

# Paso 5
#=======
# Reconstruir la imagen original
def reconstruct(laplacian_pyr):
    laplacian_top = laplacian_pyr[0]
    laplacian_lst = [laplacian_top]
    num_levels = len(laplacian_pyr) - 1
    for i in range(num_levels):
        size = (laplacian_pyr[i + 1].shape[1], laplacian_pyr[i + 1].shape[0])
        laplacian_expanded = cv.pyrUp(laplacian_top, dstsize=size)
        laplacian_top = cv.add(laplacian_pyr[i+1], laplacian_expanded)
        laplacian_lst.append(laplacian_top)
    return laplacian_lst

In [8]:
# Paso 1
#=======
# Cargamos las imágenes
img1 = cv.imread('justiceleague.jpg')
img2 = cv.imread('guason.png')
# Modificamos los tamaños de las imágenes si fuese necesario (en nuestro caso tienen el mismo tamaño)
#img1 = cv.resize(img1, (1920, 1080))
#img2 = cv.resize(img2, (1920, 1080))

# Muestro las imágenes para elegir los límites de la máscara
plt.imshow(img1)
plt.figure()
plt.imshow(img2)

In [8]:
# Crear la máscara
#mask = cv.imread('justiceleague_mask.png')
mask = np.zeros((1080,1920,3), dtype='float32')
mask[130:430,605:825,:] = (1,1,1)

num_levels = 7

# Para la 1er imagen, calcular Gaussiana y Laplaciana
gaussian_pyr_1 = gaussian_pyramid(img1, num_levels)
laplacian_pyr_1 = laplacian_pyramid(gaussian_pyr_1)
# Para la 2da imagen, calcular Gaussiana y Laplaciana
gaussian_pyr_2 = gaussian_pyramid(img2, num_levels)
laplacian_pyr_2 = laplacian_pyramid(gaussian_pyr_2)
# Calcular la pirámide Gaussiana para la máscara e invertirla
mask_pyr_final = gaussian_pyramid(mask, num_levels)
mask_pyr_final.reverse()
# Armar pirámide de fusión de las imágenes
add_laplace = blend(laplacian_pyr_1,laplacian_pyr_2,mask_pyr_final)
# Reconstruir la imagen
final  = reconstruct(add_laplace)

# Guardo en disco
cv.imwrite('fusion.png',final[num_levels])

True